# 如何一起编写提示 How to compose prompts together
LangChain 提供了一个用户友好的接口，用于将提示的不同部分组合在一起。  
您可以使用字符串提示或聊天提示来执行此操作。以这种方式构建提示可以轻松重用组件。

## 字符串提示组合
使用字符串提示时，每个模板都会连接在一起。您可以直接使用提示或字符串（列表中的第一个元素必须是prompt）。

In [2]:
from langchain_core.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("给我讲一个关于{topic}的笑话")
    + ", 讲得有趣一点"
    + "\n\n用{language}"
)

prompt

PromptTemplate(input_variables=['language', 'topic'], template='给我讲一个关于{topic}的笑话, 讲得有趣一点\n\n用{language}')

In [3]:
prompt.format(topic="体育", language="英文")

'给我讲一个关于体育的笑话, 讲得有趣一点\n\n用英文'

## 聊天提示组合
聊天提示由一串消息组成。与上述示例类似，我们可以连接聊天提示模板。每个新元素都是最终提示中的一条新消息。

首先，让我们用 SystemMessage 初始化 ChatPromptTemplate。

In [4]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

prompt = SystemMessage(content="你是个好海盗")

然后，您可以轻松创建一个管道，将其与其他消息或消息模板相结合。  
当没有要格式化的变量时，使用消息；当有要格式化的变量时，使用MessageTemplate。  
您也可以只使用字符串（注意：这将自动推断为 HumanMessagePromptTemplate。）

In [5]:
new_prompt = (
    prompt + HumanMessage(content="你好") + AIMessage(content="什么事?") + "{input}"
)

在底层，这会创建 ChatPromptTemplate 类的一个实例，因此您可以像以前一样使用它！

In [7]:
new_prompt.format_messages(input="我说嗨")

[SystemMessage(content='你是个好海盗'),
 HumanMessage(content='你好'),
 AIMessage(content='什么事?'),
 HumanMessage(content='我说嗨')]

## 使用 PipelinePrompt
LangChain 包含一个名为 PipelinePromptTemplate 的类，当您想要重复使用提示的部分内容时，该类非常有用。PipelinePrompt 由两个主要部分组成：

 - Final prompt最终提示：返回的最终提示
 - Pipeline prompts管道提示：由字符串名称和提示模板组成的元组列表。每个提示模板都将被格式化，然后作为具有相同名称的变量传递给未来的提示模板。

In [9]:
from langchain_core.prompts import PipelinePromptTemplate, PromptTemplate

full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """您正在冒充{person}。"""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """以下是一个交互示例：

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """现在，真正地做这件事！

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

pipeline_prompt.input_variables

['person', 'example_q', 'input', 'example_a']

In [10]:
print(
    pipeline_prompt.format(
        person="伊隆·马斯克",
        example_q="你最喜欢的车是什么?",
        example_a="特斯拉",
        input="你最喜欢的社交媒体网站是什么?",
    )
)

您正在冒充伊隆·马斯克。

以下是一个交互示例：

Q: 你最喜欢的车是什么?
A: 特斯拉

现在，真正地做这件事！

Q: 你最喜欢的社交媒体网站是什么?
A:
